In [34]:
import read_write_qrcd as q_reader
from gensim import corpora, models, similarities
from collections import defaultdict
import pprint

In [5]:
train_data_file = '../quranqa/datasets/qrcd_v1.1_train.jsonl'
train = q_reader.load_jsonl(train_data_file)
train[0]

Loaded 710 records from ../quranqa/datasets/qrcd_v1.1_train.jsonl


{'pq_id': '2:8-16_364',
 'passage': 'ومن الناس من يقول آمنا بالله وباليوم الآخر وما هم بمؤمنين. يخادعون الله والذين آمنوا وما يخدعون إلا أنفسهم وما يشعرون. في قلوبهم مرض فزادهم الله مرضا ولهم عذاب أليم بما كانوا يكذبون. وإذا قيل لهم لا تفسدوا في الأرض قالوا إنما نحن مصلحون. ألا إنهم هم المفسدون ولكن لا يشعرون. وإذا قيل لهم آمنوا كما آمن الناس قالوا أنؤمن كما آمن السفهاء ألا إنهم هم السفهاء ولكن لا يعلمون. وإذا لقوا الذين آمنوا قالوا آمنا وإذا خلوا إلى شياطينهم قالوا إنا معكم إنما نحن مستهزئون. الله يستهزئ بهم ويمدهم في طغيانهم يعمهون. أولئك الذين اشتروا الضلالة بالهدى فما ربحت تجارتهم وما كانوا مهتدين.',
 'surah': 2,
 'verses': '8-16',
 'question': 'لماذا سيُحاسب ويُعذب الضال يوم القيامة ان كان ""من يضلل الله فما له من هاد"" كما ورد من قوله تعالى في آية 23 و آية 36 من سورة الزمر؟',
 'answers': [{'text': 'أولئك الذين اشتروا الضلالة بالهدى', 'start_char': 504}]}

In [48]:
verses = [ver.strip() for ver in train[0].get('passage').split('.') if ver.strip()]
question = train[0].get('question')
print('question:')
print(question)
print('verses:')
for ver in verses:
    print(ver)
answers = train[0].get('answers')
print(answers)

question:
لماذا سيُحاسب ويُعذب الضال يوم القيامة ان كان ""من يضلل الله فما له من هاد"" كما ورد من قوله تعالى في آية 23 و آية 36 من سورة الزمر؟
verses:
ومن الناس من يقول آمنا بالله وباليوم الآخر وما هم بمؤمنين
يخادعون الله والذين آمنوا وما يخدعون إلا أنفسهم وما يشعرون
في قلوبهم مرض فزادهم الله مرضا ولهم عذاب أليم بما كانوا يكذبون
وإذا قيل لهم لا تفسدوا في الأرض قالوا إنما نحن مصلحون
ألا إنهم هم المفسدون ولكن لا يشعرون
وإذا قيل لهم آمنوا كما آمن الناس قالوا أنؤمن كما آمن السفهاء ألا إنهم هم السفهاء ولكن لا يعلمون
وإذا لقوا الذين آمنوا قالوا آمنا وإذا خلوا إلى شياطينهم قالوا إنا معكم إنما نحن مستهزئون
الله يستهزئ بهم ويمدهم في طغيانهم يعمهون
أولئك الذين اشتروا الضلالة بالهدى فما ربحت تجارتهم وما كانوا مهتدين
[{'text': 'أولئك الذين اشتروا الضلالة بالهدى', 'start_char': 504}]


In [12]:
stoplist = ''.split()
texts = [
    [word for word in ver.split() if word not in stoplist]
    for ver in verses
]

In [17]:
print(texts)

[['ومن', 'الناس', 'من', 'يقول', 'آمنا', 'بالله', 'وباليوم', 'الآخر', 'وما', 'هم', 'بمؤمنين'], ['يخادعون', 'الله', 'والذين', 'آمنوا', 'وما', 'يخدعون', 'إلا', 'أنفسهم', 'وما', 'يشعرون'], ['في', 'قلوبهم', 'مرض', 'فزادهم', 'الله', 'مرضا', 'ولهم', 'عذاب', 'أليم', 'بما', 'كانوا', 'يكذبون'], ['وإذا', 'قيل', 'لهم', 'لا', 'تفسدوا', 'في', 'الأرض', 'قالوا', 'إنما', 'نحن', 'مصلحون'], ['ألا', 'إنهم', 'هم', 'المفسدون', 'ولكن', 'لا', 'يشعرون'], ['وإذا', 'قيل', 'لهم', 'آمنوا', 'كما', 'آمن', 'الناس', 'قالوا', 'أنؤمن', 'كما', 'آمن', 'السفهاء', 'ألا', 'إنهم', 'هم', 'السفهاء', 'ولكن', 'لا', 'يعلمون'], ['وإذا', 'لقوا', 'الذين', 'آمنوا', 'قالوا', 'آمنا', 'وإذا', 'خلوا', 'إلى', 'شياطينهم', 'قالوا', 'إنا', 'معكم', 'إنما', 'نحن', 'مستهزئون'], ['الله', 'يستهزئ', 'بهم', 'ويمدهم', 'في', 'طغيانهم', 'يعمهون'], ['أولئك', 'الذين', 'اشتروا', 'الضلالة', 'بالهدى', 'فما', 'ربحت', 'تجارتهم', 'وما', 'كانوا', 'مهتدين']]


In [19]:
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

In [22]:
tfidf = models.TfidfModel(corpus) 
corpus_tfidf = tfidf[corpus]
lsi_model = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=2)
corpus_lsi = lsi_model[corpus_tfidf]

In [27]:
q_bow = dictionary.doc2bow(question.split())
vec_lsi = lsi_model[q_bow]
print(q_bow)  
print(vec_lsi)

[(5, 3), (14, 1), (23, 1), (47, 1), (68, 1)]
[(0, 0.5729891824272499), (1, -0.3265007886390873)]


In [37]:
lsi_index = similarities.MatrixSimilarity(corpus_lsi)
tfidf_index = similarities.MatrixSimilarity(corpus_tfidf)

In [38]:
lsi_sims = lsi_index[vec_lsi]
tfidf_sims = tfidf_index[q_bow]

In [42]:
def sort_sims(sims, docs):
    result = []
    sims = sorted(enumerate(sims), key=lambda item: -item[1])
    for doc_position, doc_score in sims:
        result.append( (doc_score, docs[doc_position]) )
    return result

In [46]:
print(question)
sort_sims(lsi_sims, verses)

لماذا سيُحاسب ويُعذب الضال يوم القيامة ان كان ""من يضلل الله فما له من هاد"" كما ورد من قوله تعالى في آية 23 و آية 36 من سورة الزمر؟


[(0.9975539,
  'أولئك الذين اشتروا الضلالة بالهدى فما ربحت تجارتهم وما كانوا مهتدين'),
 (0.97135884,
  'وإذا قيل لهم آمنوا كما آمن الناس قالوا أنؤمن كما آمن السفهاء ألا إنهم هم السفهاء ولكن لا يعلمون'),
 (0.96080565, 'ألا إنهم هم المفسدون ولكن لا يشعرون'),
 (0.93313026, 'ومن الناس من يقول آمنا بالله وباليوم الآخر وما هم بمؤمنين'),
 (0.8523612, 'يخادعون الله والذين آمنوا وما يخدعون إلا أنفسهم وما يشعرون'),
 (0.77143127,
  'في قلوبهم مرض فزادهم الله مرضا ولهم عذاب أليم بما كانوا يكذبون'),
 (0.6204605, 'الله يستهزئ بهم ويمدهم في طغيانهم يعمهون'),
 (0.34390667, 'وإذا قيل لهم لا تفسدوا في الأرض قالوا إنما نحن مصلحون'),
 (0.23883024,
  'وإذا لقوا الذين آمنوا قالوا آمنا وإذا خلوا إلى شياطينهم قالوا إنا معكم إنما نحن مستهزئون')]

In [47]:
print(question)
sort_sims(tfidf_sims, verses)

لماذا سيُحاسب ويُعذب الضال يوم القيامة ان كان ""من يضلل الله فما له من هاد"" كما ورد من قوله تعالى في آية 23 و آية 36 من سورة الزمر؟


[(0.28645146, 'ومن الناس من يقول آمنا بالله وباليوم الآخر وما هم بمؤمنين'),
 (0.1305212,
  'وإذا قيل لهم آمنوا كما آمن الناس قالوا أنؤمن كما آمن السفهاء ألا إنهم هم السفهاء ولكن لا يعلمون'),
 (0.11826248, 'الله يستهزئ بهم ويمدهم في طغيانهم يعمهون'),
 (0.09150341,
  'أولئك الذين اشتروا الضلالة بالهدى فما ربحت تجارتهم وما كانوا مهتدين'),
 (0.08784388,
  'في قلوبهم مرض فزادهم الله مرضا ولهم عذاب أليم بما كانوا يكذبون'),
 (0.057216108, 'وإذا قيل لهم لا تفسدوا في الأرض قالوا إنما نحن مصلحون'),
 (0.05253224, 'يخادعون الله والذين آمنوا وما يخدعون إلا أنفسهم وما يشعرون'),
 (0.0, 'ألا إنهم هم المفسدون ولكن لا يشعرون'),
 (0.0,
  'وإذا لقوا الذين آمنوا قالوا آمنا وإذا خلوا إلى شياطينهم قالوا إنا معكم إنما نحن مستهزئون')]

In [49]:
answers

[{'text': 'أولئك الذين اشتروا الضلالة بالهدى', 'start_char': 504}]